# Segmenting and Clustering Neighbourhoods in Toronto

### Part 1: Scraping data of Postal codes in Toronto from a wikipedia page and wrangling the data into a pandas dataframe.

In [1]:
#importing the libraries
import pandas as pd
import numpy as np


In [2]:
import requests #library to handle requests
from bs4 import BeautifulSoup #library to parse HTML documents

In [3]:
#importing the data from the wikipedia page
url_data='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url_data) #to send HTTPS request
print(response.status_code)

200


In [4]:
#parsing the data from HTML into a beautifulsoup object
soup=BeautifulSoup(response.text,'html.parser')
toronto_data=soup.find('table',{'class':"wikitable"})
toronto_data

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [5]:
#converting the Wikitable to a DataFrame
df_toronto_data=pd.read_html(str(toronto_data)) #made a list
df_toronto_data=pd.DataFrame(df_toronto_data[0]) #converting list to DataFrame
df_toronto_data.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
#drop the rows which do not have Borough assigned to them. 
i=df_toronto_data[(df_toronto_data.Borough=='Not assigned')].index
df_toronto_data.drop(i)
df_toronto_data.reset_index(drop=True, inplace=True)
df_toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#replace the neighbourhoods that are not assigned with their Borough name
for j in enumerate(df_toronto_data):
    if ('Neighbourhood' =='Not assigned'):
        df_toronto_data['Neighbourhood'].replace('Not assigned',df_toronto_data.loc[j,'Borough'],inplace=True)

In [13]:
df_toronto_data.head(5)   

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [31]:
#printing the number of rows in the dataframe
print('The number of rows in the dataframe are {}'.format(df_toronto_data.shape[0]))

The number of rows in the dataframe are 103


### Part 2: Locating the latitudes and longitudes of the postal codes

In [14]:
#importing the co-ordinates dataframe
csvpath_geocords='http://cocl.us/Geospatial_data'
df_geo_cords=pd.read_csv(csvpath_geocords)

In [24]:
df_geo_cords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
#merging the co-ordinates dataframe and the main postal dataframe
df_toronto=pd.merge(left=df_toronto_data,right=df_geo_cords,left_on='Postal Code',right_on='Postal Code')
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 3: Exploring Neighbourhoods in Toronto

Import necessary libraries

In [30]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Use the geopy library to get the geographical co-ordinates of Toronto

In [31]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with Boroughs

In [33]:
toronto_map=folium.Map(location=[latitude,longitude], zoom_start=10)
for Latitude, Longitude, Borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

I am now going to explore the various neighbourhoods of Toronto

Firstly, I am going to list down all the boroughs in Toronto

In [35]:
df_toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

I wish to explore Downtown Toronto. To do so, I use Foursquare. 

In [70]:
CLIENT_ID = '0Z3GNZ2WYZX2WRSGAI1UBMC4ZRX3NSELLLVQ5XMVGCFQHC5B' # your Foursquare ID
CLIENT_SECRET = 'CY15FQM3VWOTWMODDOLVS1ZZVOBA3CDXUGFL35NCYWSS0W0X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


Create another dataframe for Downtown Toronto

In [38]:
df_downtown_toronto=df_toronto[df_toronto['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_downtown_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


To explore venues around the University of Toronto. So, I will firstly define the geographical co-ordinates of this location and start looking for places to explore  

In [95]:
n=df_downtown_toronto[df_downtown_toronto['Neighbourhood'].str.contains('University of Toronto')].index
university_latitude=df_downtown_toronto.loc[n,'Latitude'].values[0]
university_longitude=df_downtown_toronto.loc[n,'Longitude'].values[0]
university_ll=university_latitude,university_longitude
university_ll

(43.6626956, -79.4000493)

In [107]:
radius=1000 #in meters
url_downtown_toronto='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,university_latitude,university_longitude,radius)
results = requests.get(url_downtown_toronto).json()
url_downtown_toronto
results

{'meta': {'code': 200, 'requestId': '5fa0565a2514044861308b08'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 158,
  'suggestedBounds': {'ne': {'lat': 43.67169560900001,
    'lng': -79.38763155581663},
   'sw': {'lat': 43.65369559099999, 'lng': -79.41246704418339}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '527d450111d25050de4ea0d8',
       'name': 'Rasa',
       'location': {'address': '196 Robert Street',
        'lat': 43.662756751275445,
        'lng': -79.40398803188654,
        'labeledLatLngs': [{'label': 'display',
     

Clean the json file and convert the data into a pandas dataframe. 

In [108]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [109]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-109-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rasa,Restaurant,43.662757,-79.403988
1,Yasu,Japanese Restaurant,43.662837,-79.403217
2,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
3,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703
4,Piano Piano,Italian Restaurant,43.662949,-79.402898


The total categories returned around the University of Toronto

In [111]:
nearby_venues['categories'].unique()

array(['Restaurant', 'Japanese Restaurant', 'Dessert Shop', 'Beer Bar',
       'Italian Restaurant', 'Bakery', 'French Restaurant', 'Bar',
       'Sushi Restaurant', 'Park', 'Vegetarian / Vegan Restaurant',
       'Concert Hall', 'Hostel', 'Yoga Studio', 'Hotel', 'Music School',
       'Supermarket', 'Museum', 'Thai Restaurant', 'Gaming Cafe', 'Café',
       'Bookstore', 'Gift Shop', 'Comfort Food Restaurant', 'College Gym',
       'Doner Restaurant'], dtype=object)

In [112]:
print('There are {} venues around the University of Toronto'.format(nearby_venues.shape[0]))

There are 30 venues around the University of Toronto


Now, exploring all the neighbourhoods around Downtown Toronto

In [114]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

Run the function on each neighbourhood and create a new dataframe

In [116]:
downtown_venues=getNearbyVenues(names=df_downtown_toronto['Neighbourhood'],latitudes=df_downtown_toronto['Latitude'],longitudes=df_downtown_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [117]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Checking how many venues were returned for each neighbourhood

In [126]:
print('The total venues returned around downtown Toronto are {}'.format(downtown_venues.shape[0]))
print('There are {} unique categories returned in downtown Toronto'.format(len(downtown_venues['Venue Category'].unique())))

The total venues returned around downtown Toronto are 516
There are 151 unique categories returned in downtown Toronto


In [122]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
"First Canadian Place, Underground city",30,30,30,30,30,30
"Garden District, Ryerson",30,30,30,30,30,30
"Harbourfront East, Union Station, Toronto Islands",30,30,30,30,30,30
